# LEIA-ChipWhisperer Quickstart

**SUMMARY**: *In this lab, you'll learn how to setup and connect to your ChipWhisperer hardware. We'll also cover how to build firmware the LEIA board and target firmware, how to capture power traces, and how to communicate with target devices.*

**LEARNING OUTCOMES:**
* Setting up LEIA Hardware
* Using the smartleia Python API to connect to your hardware
* Communication with the target
* Capturing a power trace

## Prerequisites

Hold up - before continuing, ensure you have done the following:

* ☑ Clone and install the H2LAB ChipWhisperer repo (https://github.com/h2lab/chipwhisperer)
* ☑ Run through the Jupyter introduction.
* ☑ Run through the LEIA Solo introduction.
* ☑ Update your ChipWhisperer Firmware 

## Physical Setup

### Board pinout

![LEIA Solo Pinout](https://h2lab.org/images/devices/leia_pinout.png "LEIA Pinout")


### Testing points

![LEIA Solo test points TOP](https://h2lab.org/images/devices/leia_test_points_top.png)

![LEIA Solo test points BOT](https://h2lab.org/images/devices/leia_test_points_bot.png)

### Setting up measure Mode

In order to setup LEIA in the measure mode: 

 * Move the PRG1, PRG2, PRG3 (LEIA Solo < v1.4) and PRG4 to the LEIA position (1-2).
 * Remove the shunt bypass jumber if it is set.
 * Move the tearing jumper to the OFF position.
 * Setup the power source for the smartcard. We would advise an external "clean" power source for clean measurements. However, we are able to get proper traces with the USB-C power supply on the funcard. 

## Clonning the ChipWhisperer LEIA SOLO repository

As the LEIA target has not been (yet) integrated to the Chipwhisperer upstream repository, you will have to install our specific fork from here: https://github.com/h2lab/chipwhisperer


In [ ]:
!git clone https://github.com/h2lab/chipwhisperer
!cd chipwhisperer && python3 -m pip install -e . --user

## Communication with the Target using the ChipWhisperer

Communication with targets is done through the ChipWisperer SDK.

First you need to import boath smartleia and the chipwhisperer modules

In [ ]:
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import chipwhisperer as cw
from smartleia import APDU, TriggerPoints, T

## Connecting to ChipWhisperer

Now that your hardware is all setup, we can now learn how to connect to it. We can connect to the ChipWhisperer with:

In [ ]:
scope = cw.scope()

### Configuring the chipwhisperer scope

In [ ]:
# setup scope parameters
scope.adc.samples = 10000

### Connecting to the target device is simple as well:

In [ ]:
target = cw.target(scope, cw.targets.LeiaTarget) 

### Initialize the LEIA reader, 

Initialize the LEIA reader and wait for the smartcard to be inserted then request the ATR.


In [ ]:
import itertools, sys
spinner = itertools.cycle(['-', '/', '|', '\\'])

def wait_for_card(target):
    """
        Wait for the smartcard to be inserted
    """
    print('Waiting for card to be inserted...\t',end='')
    while not(target.is_card_inserted()):
        sys.stdout.write(next(spinner))
        sys.stdout.flush()            
        sys.stdout.write('\b')
    print('OK')
    

wait_for_card(target)


In [ ]:
target.reset()

### Initialize a connection to a card

In [ ]:
target.init()
target.configure_smartcard(
        protocol_to_use = T.T0,
        negotiate_pts = False)
target.select_applet(applet=[0x45, 0x75, 0x74, 0x77, 0x74, 0x75, 0x36, 0x41, 0x70, 0x80])

### Sending APDU to the target 

In [ ]:
apdu_select = APDU(cla=0x00, ins=0x01, p1=0x00, p2=0x00)
apdu_select
APDU(cla=0x0, ins=0x1, p1=0x0, p2=0x0, lc=0, le=0, send_le=1)
resp = target.send_APDU(apdu_select)
print(resp)
target.close()

### Define Trigger Strategies 

Setting a LEIA trigger strategy that point just before sending a simple APDU

In [ ]:
target.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU],delay=0)

print(target.get_trigger_strategy(1))

### Setting the inputs

In [ ]:
AES_TEST_VECTOR_KEY = "2b7e151628aed2a6abf7158809cf4f3c"
AES_TEST_VECTOR_DATA = "6bc1bee22e409f96e93d7e117393172a"
AES_TEST_VECTOR_CIPHER = "3ad77bb40d7a3660a89ecaf32466ef97"

# Seting the key
key = AES_TEST_VECTOR_KEY
plaintext = AES_TEST_VECTOR_DATA

target.loadEncryptionKey(key)
target.loadInput(plaintext)

### Checking data on the target

In [ ]:
target.checkEncryptionKey(key)plaintext

In [ ]:
target.checkPlaintext(plaintext)

## Capturing Traces

Now that the target's programmed and we know how to communicate with it, let's start recording some power traces! To capture a trace:

1. Arm the ChipWhisperer with `scope.arm()`. It will begin capturing as soon as it is triggered (which in our case is a rising edge on `gpio4`.
1. `scope.capture()` will read back the captured power trace, blocking until either ChipWhisperer is done recording, or the scope times out. Note that the error return will tell you whether or not the scope timed out. It does not return the captured scope data.
1. You can read back the captured power trace with `scope.get_last_trace()`.


In [ ]:
# arm the scope
scope.arm()

# launch computation
target.go()

# wait for target to finish
while target.isDone() is False and timeout:
    timeout -= -1
    time.sleep(0.01)

try:
    ret = scope.capture()
    if ret:
        print("Timeout happened during acquisition")
except IOError as e:
    print(f"IOError: {e}")

cipher = target.readOutput().data

# print the result value
cipherstr = ''.join('{:02x}'.format(x) for x in cipher)

# print the expected value
expectedstr=''.join('{:02x}'.format(x) for x in target.getExpected())
print(expectedstr, cipherstr)
   



## Showing the traces

In [ ]:
import matplotlib.pyplot as plt

trace = scope.get_last_trace()  

if trace:
    plt.plot(trace)
    plt.show()
else:
    print("No trace captured")

## Saving the traces

In [ ]:
import numpy as np
from datetime import datetime
from tqdm import trange
import os

now = datetime.now()
fmt_string = "{:02}{:02}_{}.npy"

traces = []     # captured traces array
datain = []     # testcase data input array
known_keys = [] # testcase key input array

traces.append(trace)
datain.append(plaintext)
known_keys.append(key)

trace_array = np.asarray(traces)
datain_array = np.asarray(datain)
known_keys = np.asarray(known_keys)

trace_file_path = os.path.join(fmt_string.format(now.hour, now.minute, "traces"))
datain_file_path = os.path.join(fmt_string.format(now.hour, now.minute, "data"))
keys_file_path = os.path.join(fmt_string.format(now.hour, now.minute, "keys"))

np.save(trace_file_path, trace_array)
np.save(datain_file_path, datain_array)
np.save(keys_file_path, known_keys)


## Conclusion

And that's it! 
As a final step, we should disconnect from the hardware so it doesn't stay "in use" by this notebook.

In [ ]:
scope.dis()
target.dis()